In [1]:
# Notebook to compare basic meteorological parameters between WRF simulations and ERA5 data
# Some of the following imports are not used right now, but will retain for future flexibility
import os
import subprocess
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.cbook as cbook
from matplotlib.colors import Normalize
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib.colors as mcolors
import pandas as pd
import xarray as xr
import numpy as np
import math
from numpy import *
from pylab import *
import pygrib
import pyproj

from siphon.catalog import TDSCatalog
from siphon.http_util import session_manager
from datetime import datetime, timedelta
from xarray.backends import NetCDF4DataStore
from netCDF4 import Dataset
import metpy as metpy
import metpy.calc as mpcalc
from metpy.plots import ctables
from metpy.units import units
from metpy.plots import add_metpy_logo, add_timestamp
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from cartopy import config
import wrf
from wrf import (to_np, interplevel, geo_bounds, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                          facecolor="none", name="admin_1_states_provinces_shp")
import glob

In [2]:
"""
Data user settings, constants, and level ranges
"""
################## wrfout files path ###################################################

thompson_aa_38 = "/scratch/sawyer/wwrf/2017-01-09/thompson_aa_38/wrfout_d01*"
################### Saving plots option ##################

save = True

########### If you want to save plots, specify the subdirectory you want them in ###########

plotsdir = "/home/jupyter-sdsmit12@ncsu.edu/WRF_project_1/wwrf/2017-01-09/plots/"
print(plotsdir)
############ Gravity ###########
g0 = 9.80665

################ Set bounds for plotting #################

wlon = -160
elon = -115
slat = 15
nlat = 50

############################# Do we only want to plot IVT? ###############################

compute_ivt = True

############################# Set ranges and intervals for plotting ###############################


levels_1000 = np.arange(-180,180,30)
q_levels = np.arange(0, 22, 1)
wnd_levels_1000 = np.arange(0, 30, 1)

levels_850 = np.arange(1000, 2000, 50)
q_levels = np.arange(0, 16, .5)
wnd_levels_850 = np.arange(20, 70, 5)    

levels_700 = np.arange(2350, 3150, 30)
q_levels = np.arange(0, 12, .5)
wnd_levels_700 = np.arange(20, 65, 1) 

levels_500 = np.arange(4000, 6000, 60)
q_levels = np.arange(0, 16, .5)
wnd_levels_500 = np.arange(20, 120, 1) 

levels_300 = np.arange(8200, 9600, 120)
wnd_levels_300 = np.arange(20, 180, 1) 

levels_200 = np.arange(10800,12300, 120)
wnd_levels_200 = np.arange(20, 160, 10) 

ivt_levels = np.arange(250,1150,1)
pv_levels = np.arange(0,1,.1)
pmsl_levels = np.arange(960, 1060, 4)


########### Define the colors in the desired sequence for IVT: ###########

colors = ['#FFFF00', '#FFEE00','#FFDC00', '#FFB700',
          '#FFA300', '#FF9000', '#FF7D00', '#FF6800',
          '#FF5200', '#C70039','#900C3F', (.88,.24,.69)]

############## Create a colormap using ListedColormap #################

cmap = mcolors.ListedColormap(colors)

########## Create a list of Forecast Hours for plotting ##############

fcst = list(arange(0, 171,3))


/home/jupyter-sdsmit12@ncsu.edu/WRF_project_1/wwrf/2017-01-09/plots/


In [3]:
"""
This function computes Integrated Vapor Transport (IVT) from wrfout files
"""
def calculate_IVT(ua,va,p,mr):
   
    
    uflux_l = []
    vflux_l = []
    for m in range(0,len(mr)-2):
        layer_diff = p[m,:,:]-p[m+1,:,:]
        ql = (mr[m+1,:,:]+mr[m,:,:])/2
        ul = (ua[m+1,:,:]+ua[m,:,:])/2
        vl = (va[m+1,:,:]+va[m,:,:])/2
        qfl = (ql/9.80665)*layer_diff
        uflux= ul * qfl
        vflux = vl * qfl
        uflux_l.append(uflux)
        vflux_l.append(vflux)

    uflux_l=np.asarray(uflux_l)
    vflux_l=np.asarray(vflux_l)
    uIVT = np.sum(uflux_l, axis = 0)
    vIVT = np.sum(vflux_l, axis = 0)
    IVT_tot=np.sqrt(uIVT**2+vIVT**2)
    #IVT_tot=xr.DataArray(IVT_tot,dims=['lat','lon'])
    
    return IVT_tot

In [4]:
# Prepare files for processing
datafiles = (glob.glob(thompson_aa_38))
datafiles.sort()


numfiles=len(datafiles)
print(numfiles)

57


In [5]:
for i in range(0,numfiles):

    ncfile = Dataset(datafiles[i])
    Time=wrf.extract_times(ncfile, timeidx=0, method='cat', squeeze=True, cache=None, meta=False, do_xtime=False)
    timestr=(str(Time))
    
    # Set up one time string for plot titles, another for file names
    titletime=(timestr[0:10]+' '+timestr[11:16])
    filetime=(timestr[0:10]+'_'+timestr[11:13])
    print('WRF valid time: ',filetime)
    plot_filetime = (timestr[0:10]+' '+timestr[11:13])
    dates = int(timestr[0:10].replace('-', ''))
    
    # Get all the variables we need from wrf
    
    p = getvar(ncfile, "pressure")
    wspd = getvar(ncfile, "wspd_wdir",units='kts')[0,:]
    height = getvar(ncfile, "z")
    slp = getvar(ncfile, "slp")
    # Smooth the sea level pressure since it tends to be noisy near complex terrain
    smooth_slp = smooth2d(slp, 3, cenweight=4)
    windsp_850 = interplevel(wspd, p, 850)
    heights_850 = interplevel(height, p, 850)    
    windsp_500 = interplevel(wspd, p, 500)
    heights_500 = interplevel(height, p, 500) 
    windsp_250 = interplevel(wspd, p, 200)
    heights_250 = interplevel(height, p, 200) 
    # Get the lat/lon coordinates 
    lats, lons = latlon_coords(p)

    # fix lons around dateline from wrf output
    new_lons = np.where(lons > 0, lons - 360, lons)

    # Set up the plot
    cart_proj = get_cartopy(p)

    ua = getvar(ncfile, "ua")
    va = getvar(ncfile, "va")
    p = p * 100
    mr = ncfile['QVAPOR'][0,:,:,:]
    ivt = calculate_IVT(ua,va,p,mr)
    

    fig, axes = plt.subplots(2,2,subplot_kw={'projection': cart_proj},figsize=(24.5, 17.))

    # dark brown for state/coastlines
    dark_brown = (0.4, 0.2, 0)

    ################################################################################
    # Upper-Left Panel

    axes[0,0].set_extent([wlon,elon,slat,nlat])
    axes[0,0].add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor=dark_brown)#Add coastlines)
    axes[0,0].add_feature(cfeature.STATES, linewidth=0.5,edgecolor=dark_brown) #Add US states
    gl = axes[0,0].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    gl.top_labels=False   # suppress top labels
    gl.right_labels=False 

    # Plot the contours
    cs = axes[0,0].contour(to_np(new_lons), to_np(lats), to_np(smooth_slp), levels=pmsl_levels, colors='black',linewidths=0.9,transform=ccrs.PlateCarree())
    axes[0,0].clabel(cs, fmt='%1.0f', inline=True,levels=pmsl_levels)

    cs1 = axes[0,0].contourf(to_np(new_lons), to_np(lats), to_np(ivt), cmap = cmap, levels = ivt_levels, transform=ccrs.PlateCarree())

    #Add a colorbar
    ivt_cbar = plt.colorbar(cs1, orientation = 'horizontal', shrink = 0.8)
    ivt_cbar.set_label("IVT (kg $m^{-1}$$s^{-1}$)", fontsize = 12)

    # Set the plot title
    axes[0,0].set_title(f'Thompson AA MP Scheme \nSea Level Pressure (hPa) \nIVT (kg $m^{-1}$$s^{-1}$)', fontsize=12,loc='left')

    axes[0,0].set_title(f'VALID: {plot_filetime} UTC \n Forecast Hour: {fcst[i]}',fontsize=12,loc='right')

    ################################################################################
    # Upper-Right Panel
    axes[0,1].set_extent([wlon,elon,slat,nlat])
    axes[0,1].add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor=dark_brown)#Add coastlines)
    axes[0,1].add_feature(cfeature.STATES, linewidth=0.5,edgecolor=dark_brown) #Add US states
    gl = axes[0,1].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    gl.top_labels=False   # suppress top labels
    gl.right_labels=False 

    # Plot the contours
    cs_850 = axes[0,1].contour(to_np(new_lons), to_np(lats), to_np(heights_850), levels=levels_850, colors='black',linewidths=0.9,transform=ccrs.PlateCarree())
    axes[0,1].clabel(cs_850, fmt='%1.0f', inline=True)

    cs2_850 = axes[0,1].contourf(to_np(new_lons), to_np(lats), to_np(windsp_850), cmap = 'jet', levels = wnd_levels_850, transform=ccrs.PlateCarree())

    #Add a colorbar
    wind_cbar = plt.colorbar(cs2_850, orientation = 'horizontal', shrink = 0.8)
    wind_cbar.set_label("wind speed (kt)", fontsize = 12)

    # Set the plot title
    axes[0,1].set_title(f'Thompson AA MP Scheme \n850 hPa Geopotential Heights (m) \nWinds (kt), \n ',fontsize=12,loc='left')

    axes[0,1].set_title(f'VALID: {plot_filetime} UTC \n Forecast Hour: {fcst[i]}',fontsize=12,loc='right')

    ################################################################################
    # Bottom Left Panel
    axes[1,0].set_extent([wlon,elon,slat,nlat])
    axes[1,0].add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor=dark_brown)#Add coastlines)
    axes[1,0].add_feature(cfeature.STATES, linewidth=0.5,edgecolor=dark_brown) #Add US states
    gl = axes[1,0].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    gl.top_labels=False   # suppress top labels
    gl.right_labels=False 

    # Plot the contours
    p3_cs = axes[1,0].contour(to_np(new_lons), to_np(lats), to_np(heights_500), levels=levels_500, colors='black',linewidths=0.9,transform=ccrs.PlateCarree())
    axes[1,0].clabel(p3_cs, fmt='%1.0f', inline=True)

    p3_cs2 = axes[1,0].contourf(to_np(new_lons), to_np(lats), to_np(windsp_500), cmap = 'jet', levels = wnd_levels_500, transform=ccrs.PlateCarree())

    #Add a colorbar
    wind_cbar = plt.colorbar(p3_cs2, orientation = 'horizontal', shrink = 0.8)
    wind_cbar.set_label("wind speed (kt)", fontsize = 12)

    # Set the plot title
    axes[1,0].set_title(f'Thompson AA MP Scheme \n500 hPa Geopotential Heights (m) \nWinds (kt), \n ',fontsize=12,loc='left')

    axes[1,0].set_title(f'VALID: {plot_filetime} UTC \n Forecast Hour: {fcst[i]}',fontsize=12,loc='right')

    ################################################################################
    # Bottom-Right Panel
    axes[1,1].set_extent([wlon,elon,slat,nlat])
    axes[1,1].add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor=dark_brown)#Add coastlines)
    axes[1,1].add_feature(cfeature.STATES, linewidth=0.5,edgecolor=dark_brown) #Add US states
    gl = axes[1,1].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    gl.top_labels=False   # suppress top labels
    gl.right_labels=False 

    # Plot the contours
    p3mom_cs = axes[1,1].contour(to_np(new_lons), to_np(lats), to_np(heights_250), levels=levels_200, colors='black',linewidths=0.9,transform=ccrs.PlateCarree())
    axes[1,1].clabel(p3mom_cs, fmt='%1.0f', inline=True)

    p3mom_cs2 = axes[1,1].contourf(to_np(new_lons), to_np(lats), to_np(windsp_250), cmap = 'jet', levels = wnd_levels_200, transform=ccrs.PlateCarree())

    #Add a colorbar
    wind_cbar = plt.colorbar(p3mom_cs2, orientation = 'horizontal', shrink = 0.8)
    wind_cbar.set_label("wind speed (kt)", fontsize = 12)

    # Set the plot title
    axes[1,1].set_title(f'Thompson AA MP Scheme \n200 hPa Geopotential Heights (m) \nWinds (kt), \n ',fontsize=12,loc='left')

    axes[1,1].set_title(f'VALID: {plot_filetime} UTC \n Forecast Hour: {fcst[i]}',fontsize=12,loc='right')
    #plt.show()
    if save == True:
        # Create separate plot file and save as .png, then show and close
        outTPlotName= str(plot_filetime)+'_P3_2x_Cloud .png'
        fig.savefig(plotsdir+'/'+outTPlotName)
    plt.close(fig)

WRF valid time:  2017-01-03_00
WRF valid time:  2017-01-03_03
WRF valid time:  2017-01-03_06
WRF valid time:  2017-01-03_09
WRF valid time:  2017-01-03_12
WRF valid time:  2017-01-03_15
WRF valid time:  2017-01-03_18
WRF valid time:  2017-01-03_21
WRF valid time:  2017-01-04_00
WRF valid time:  2017-01-04_03
WRF valid time:  2017-01-04_06
WRF valid time:  2017-01-04_09
WRF valid time:  2017-01-04_12
WRF valid time:  2017-01-04_15
WRF valid time:  2017-01-04_18
WRF valid time:  2017-01-04_21
WRF valid time:  2017-01-05_00
WRF valid time:  2017-01-05_03
WRF valid time:  2017-01-05_06
WRF valid time:  2017-01-05_09
WRF valid time:  2017-01-05_12
WRF valid time:  2017-01-05_15
WRF valid time:  2017-01-05_18
WRF valid time:  2017-01-05_21
WRF valid time:  2017-01-06_00
WRF valid time:  2017-01-06_03
WRF valid time:  2017-01-06_06
WRF valid time:  2017-01-06_09
WRF valid time:  2017-01-06_12
WRF valid time:  2017-01-06_15
WRF valid time:  2017-01-06_18
WRF valid time:  2017-01-06_21
WRF vali